# Dataset

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
df = pd.read_csv('1950-2021_fires_with_elevation.csv')
x = df[['lat', 'lon', 'elevation']]
y = df['hectares']

In [40]:
x

,lat,lon,elevation
0,59.963,-128.172,690.0
1,59.318,-132.172,834.0
2,59.876,-131.922,852.0
3,59.760,-132.808,957.0
4,59.434,-126.172,429.0
...,...,...,...
15025,58.818,-122.812,403.0
15026,58.289,-121.729,392.0
15027,58.818,-122.729,443.0
15028,58.760,-122.729,301.0


In [29]:
y

0            8.0
1            8.0
2        12949.9
3          241.1
4            1.2
          ...   
15025        0.1
15026        0.1
15027        0.1
15028        1.2
15029        0.1
Name: hectares, Length: 15030, dtype: float64

In [15]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# Correlation between data

In [35]:
print(df.hectares.corr(df.lat))
print(df.hectares.corr(df.lon))
print(df.hectares.corr(df.elevation))

0.11243475800501282
-0.04525868154006124
0.006771238508371042


# Model

In [36]:
from sklearn.linear_model import LinearRegression

In [37]:
from sklearn.metrics import mean_squared_error

In [42]:
model = LinearRegression()
model.fit(x_train, y_train)

predictions_test = model.predict(x_test)
loss = np.sqrt(mean_squared_error(predictions_test, y_test))
loss

np.float64(762.9501597682411)

In [21]:
model.predict([[59.963, -128.172]])

C:\Users\ethan\PycharmProjects\fire-guard\model\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([1688.67392096])

In [22]:
model.coef_

array([177.86724302,   6.64276784])

In [23]:
model.intercept_

np.float64(-8125.362732658102)